# RAG evaluator
- [x] Metrics -> on Retriever component
- [x] Generator component -> with LLM-based judge
- [x] evaluation dataset
- [x] llm main style evaluation
- [x] RAGAS https://medium.com/data-science/evaluating-rag-applications-with-ragas-81d67b0ee31a
- [ ] Dashboard
- [ ] better source data on detailed project and information
- [ ] better golden dataset

In [2]:
import json
from base_models import TestQuestion


In [3]:
import tqdm


TEST_QUESTIONS_FILE = "../evaluation/eval_data.jsonl"

def load_test_questions() -> list[TestQuestion]:
    """
    Load test questions from a JSONL file
    """
    with open(TEST_QUESTIONS_FILE, "r", encoding="utf-8") as f:
        tests = []
        for line in f:
            data = json.loads(line.strip()) 
            tests.append(TestQuestion(**data))
        print("Loaded {} test questions".format(len(tests)))
    return tests

In [4]:
tests = load_test_questions()

Loaded 20 test questions


In [5]:
tests[0]
print(tests[0].question)
print(tests[0].ground_truth)
print(tests[0].category)
print(tests[0].keywords)

What is Beiji's current role and where does he work?
Beiji is a Software Engineer at United Overseas Bank (UOB) since Aug 2023.
career
['Beiji', 'Software Engineer', 'United Overseas Bank', 'UOB', '2023']


In [6]:
from collections import Counter
count = Counter([t.category for t in tests])
count

Counter({'rag_skill': 3,
         'skills': 3,
         'lifestyle': 3,
         'ai_engineering': 2,
         'education': 2,
         'personality': 2,
         'career': 1,
         'platform_engineering': 1,
         'achievement': 1,
         'future': 1,
         'engineering': 1})

In [7]:
from rag_retrieval import fetch_context

retrieval_results = fetch_context(tests[0].question)

print("Retrieval results:")
print(retrieval_results)



Retrieval results:
[Document(id='77fa1c7c-f3a2-40c7-98d5-510f45f6f5b8', metadata={'doc_type': 'data', 'source': 'hobby.md'}, page_content='Beiji has been playing basketball since middle school, and it remains his favorite sport. During his time at Southeast University, he served as **captain of the Transportation Department basketball team**, leading training sessions, match preparation, and team strategy. Balancing competition with'), Document(id='4a22976a-14a6-4d20-bb9b-6a93ffe4842e', metadata={'source': 'cv.md', 'doc_type': 'data'}, page_content='# LI BEIJI\n\n📞 +65 8432 9134  \n🔗 [LinkedIn]  \n📧 libeiji08121999@gmail.com  \n💻 [GitHub]  \n🌐 [Portfolio]\n\n---\n\n## SUMMARY'), Document(id='91aed67f-d81f-41e8-8ba8-2f828c7cc8a3', metadata={'doc_type': 'data', 'source': 'hobby.md'}, page_content='Beiji began weightlifting in 2020 and has trained consistently since. By 2024, he achieved personal records of **175kg squat**, **190kg deadlift**, and **120kg bench press**. He appreciates wei

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from base_models import RetrievalLLMEval
from rag_retrieval import generate_answer

LLM_EVAL_PROMPT = """
You are a helpful assistant that can answer questions about the user's CV and hobbies.
You are given a question and a context.
You need to evaluate the retrieval results based on the context.
User question:
{question}

Generated answer:
{generated_answer}

Golden answer:
{ground_truth}

Evaluation criteria:
- Accuracy: How many of the retrieval results are correct?
- Relevance: How relevant are the retrieval results to the question?
- Completeness: How complete are the retrieval results?
- Confidence: How confident are you in the retrieval results?
- Score: The average of accuracy, relevance, completeness

Return in the following format:
{{
    "accuracy": 3,
    "relevance": 2,
    "completeness": 4,
    "confidence": 0.9,
    "feedback": "The retrieval results is not relevant to the question but correct",
    "score": 3
}}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0);

def evaluate_response(test_question: TestQuestion) -> RetrievalLLMEval:
    """
    Evaluate the LLM-Response based on the retrieval results, not on the retrieval results based on the question
    """
    # get the context
    generated_answer, retrieval_results = generate_answer(test_question.question)

    # parse the messages
    system_messages = [SystemMessage(
        content=("You are an expert evaluator assessing the quality of answers. Evaluate the generated answer by comparing it to the reference answer. Only give 5/5 scores for perfect answers."
                 ))]
    user_messages = [HumanMessage(content=LLM_EVAL_PROMPT.format(question=test_question.question, generated_answer=generated_answer, ground_truth=test_question.ground_truth))]
    messages = system_messages + user_messages

    structured_llm = llm.with_structured_output(RetrievalLLMEval)
    response_LLM_eval = structured_llm.invoke(messages)

    return response_LLM_eval
    

In [9]:
result = evaluate_response(tests[0])

# Check if it's a BaseModel/RetrievalLLMEval instance
print("Type:", type(result))
print("Is RetrievalLLMEval?", isinstance(result, RetrievalLLMEval))
print("\nResult object:")
print(result)
print("\nAccess attributes:")
print(f"accuracy: {result.accuracy}")
print(f"relevance: {result.relevance}")
print(f"completeness: {result.completeness}")
print(f"score: {result.score}")
print(f"confidence: {result.confidence}")
print(f"feedback: {result.feedback}")

Type: <class 'base_models.RetrievalLLMEval'>
Is RetrievalLLMEval? True

Result object:
accuracy=4.0 relevance=4.0 completeness=3.0 confidence=0.9 feedback='The retrieval results are mostly correct and relevant, but lack the specific start date mentioned in the golden answer.' score=3.75

Access attributes:
accuracy: 4.0
relevance: 4.0
completeness: 3.0
score: 3.75
confidence: 0.9
feedback: The retrieval results are mostly correct and relevant, but lack the specific start date mentioned in the golden answer.


In [10]:
def evaluate_LLM(tests: list[TestQuestion]) -> RetrievalLLMEval:
    """
    Evaluate all the tests
    """
    results = []  
    for test in tests:
        results.append(evaluate_response(test))
    evaluation_result = RetrievalLLMEval(
        accuracy=sum([result.accuracy for result in results]) / len(results),
        relevance=sum([result.relevance for result in results]) / len(results),
        completeness=sum([result.completeness for result in results]) / len(results),
        score=sum([result.score for result in results]) / len(results),
        confidence=sum([result.confidence for result in results]) / len(results),
        feedback="This is the average of all the tests",
    )
    return evaluation_result

eval_result_LLM = evaluate_LLM(tests)

In [11]:
print("\nAverage of all the tests from LLM evaluation on LLM answer:")
print(f"accuracy: {eval_result_LLM.accuracy}")
print(f"relevance: {eval_result_LLM.relevance}")
print(f"completeness: {eval_result_LLM.completeness}")
print(f"score: {eval_result_LLM.score}")
print(f"confidence: {eval_result_LLM.confidence}")


Average of all the tests from LLM evaluation on LLM answer:
accuracy: 3.9
relevance: 3.35
completeness: 4.15
score: 3.8625
confidence: 0.89


### The metric based evals on RAG retrieval result
- MRR
- Keyword Coverage

In [12]:
def evaluate_mrr(keyword:str, retrieval_results:list) -> float:
    """
    Evaluate the MRR of the retrieval results,
    mrr = 1 -> first result contains the keyword
    mrr = 0.5 -> second result contains the keyword
    mrr = 0 -> no result contains the keyword
    """
    keyword = keyword.lower();
    for rank, result in enumerate(retrieval_results, start=1):
        if keyword in result.page_content.lower():
            return 1/rank
    return 0

In [13]:
from base_models import RetrievalEval


def evaluate_retrieval(test: TestQuestion) -> RetrievalEval:
    """
    Evaluate the retrieval results
    """

    retrieved_docs = fetch_context(test.question)
    mrr_scores = [evaluate_mrr(keyword, retrieved_docs) for keyword in test.keywords]# each keyword need to be calculated separately, so a list of scores
    avg_mrr = sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0.0

    # Calculate keyword coverage
    keywords_found = sum(1 for score in mrr_scores if score > 0)
    total_keywords = len(test.keywords)
    keyword_coverage = (keywords_found / total_keywords * 100) if total_keywords > 0 else 0.0

    return RetrievalEval(
        MRR=avg_mrr,
        keyword_coverage=keyword_coverage,
    )

def evaluate_all(tests: list[TestQuestion]) -> RetrievalEval:
    """
    Evaluate all the tests
    """
    results = []
    for test in tests:
        results.append(evaluate_retrieval(test))
    
    mrr_final = sum(result.MRR for result in results) / len(results)
    keyword_coverage_final = sum(result.keyword_coverage for result in results) / len(results)

    return RetrievalEval(
        MRR=format(mrr_final, ".2f"),
        keyword_coverage=format(keyword_coverage_final, ".2f"),
    )


In [14]:
eval_result_retrieval = evaluate_all(tests)
print("\nAverage of all the tests from retrieval evaluation:")
print(f"MRR: {eval_result_retrieval.MRR}")
print(f"Keyword Coverage: {eval_result_retrieval.keyword_coverage}% ")





Average of all the tests from retrieval evaluation:
MRR: 0.54
Keyword Coverage: 79.67% 


### RAGAS
https://medium.com/data-science/evaluating-rag-applications-with-ragas-81d67b0ee31a

In [15]:
from datasets import Dataset

questions = [test.question for test in tests]
ground_truths = [test.ground_truth for test in tests]
contexts = []
answers = []

for test in tests:
    test_answer, test_context = generate_answer(test.question)
    answers.append(test_answer)
    # each query have multiple context documents
    contexts.append([doc.page_content for doc in test_context])

dataset = Dataset.from_dict({
    "question": questions,
    "reference": ground_truths,
    "contexts": contexts,
    "answer": answers,
})

In [17]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

from rag_ingestion import embeddings

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    embeddings=embeddings,
)

df = result.to_pandas()

/var/folders/q9/87xq5q2d0sv9vrwx1n47nhvc0000gn/T/ipykernel_9246/2051731147.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/q9/87xq5q2d0sv9vrwx1n47nhvc0000gn/T/ipykernel_9246/2051731147.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/q9/87xq5q2d0sv9vrwx1n47nhvc0000gn/T/ipykernel_9246/2051731147.py:2: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import (
/var/folde

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 g

In [18]:
df.head()

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What is Beiji's current role and where does he...,[Beiji has been playing basketball since middl...,Beiji's current role is a Software Engineer at...,Beiji is a Software Engineer at United Oversea...,0.166667,1.0,1.000000,0.914340
1,What impact did Beiji make when rolling out n8n?,[- Led adoption and production rollout of **n8...,Beiji led the adoption and production rollout ...,He led the production rollout of n8n enabling ...,1.000000,1.0,1.000000,0.890352
2,What is the Prompt Template Hub that Beiji built?,[- Designed and implemented **Prompt Template ...,The Prompt Template Hub that Beiji built is a ...,It is a React and Spring Boot platform integra...,1.000000,1.0,0.666667,1.000000
3,Describe Beiji's experience with developer pla...,[- Built an in-house **Developer Portal** (Ang...,Beiji has significant experience with develope...,He built an in-house Developer Portal using An...,1.000000,1.0,1.000000,0.871991
4,What is Beiji’s strongest RAG-related project?,[Beiji began weightlifting in 2020 and has tra...,"Beiji's strongest RAG-related project is the ""...",The Best Price Notification Agent project wher...,0.000000,0.0,0.888889,0.988692
